In [6]:
import numpy as np
import pandas as pd
import os
os.chdir('/Users/Richard/Downloads/')

Creating a Recommender System using eCommerce behavior data frommulti-category cosmetic shop. The Data is five months of shopping but for the sake oflack of computing power we only used one month (December). Dataset contains 9 columns and 1.6 million unique values. Each row in the file represents an event. The events are all related to products and users. Each event is like many-to-many relationbetween products and users.

In [7]:
cos = pd.read_csv('cosmetics_data.csv')
cos.head()

,event_type,product_id,category_id,brand,price,user_id
0,cart,0,2,runail,1.19,0
1,view,1,4,runail,2.62,1
2,view,2,5,ingarden,4.44,2
3,cart,3,7,runail,0.40,0
4,remove_from_cart,4,8,metzger,3.33,3


Load the data

In [8]:
cos.describe()

,product_id,category_id,price,user_id
count,895207.000000,895207.000000,895207.000000,895207.000000
mean,393.183941,50.664065,12.501543,72626.757841
std,290.638593,55.892663,28.722981,51408.944410
min,0.000000,0.000000,0.060000,0.000000
25%,129.000000,14.000000,1.730000,26798.000000
50%,328.000000,23.000000,4.440000,64928.000000
75%,624.000000,67.000000,7.300000,114054.500000
max,999.000000,294.000000,327.780000,180881.000000


Looking quickly at some statistics of it

In [9]:
cos["event_type"].value_counts()


view                470846
cart                231056
remove_from_cart    137188
purchase             56117
Name: event_type, dtype: int64

Examining different cart events but for the scope of this project we are only looking at "purchases" which has 56,117 cases

In [10]:
cos["product_id"].value_counts()


41     35225
44     10875
217     8834
64      7637
123     7547
       ...  
170      360
527      360
969      360
628      360
53       359
Name: product_id, Length: 1000, dtype: int64

In [11]:
cos["event_type"] = cos["event_type"].astype('category')
cos.dtypes

event_type     category
product_id        int64
category_id       int64
brand            object
price           float64
user_id           int64
dtype: object

In [12]:
cosdummy = pd.get_dummies(cos[['event_type']])
cosdummy['product_id'] = cos['product_id']
cosdummy['category_id'] = cos['category_id']
cosdummy['brand'] = cos['brand']
cosdummy['price'] = cos['price']
cosdummy['user_id'] = cos['user_id']
cosdummy

,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view,product_id,category_id,brand,price,user_id
0,1,0,0,0,0,2,runail,1.19,0
1,0,0,0,1,1,4,runail,2.62,1
2,0,0,0,1,2,5,ingarden,4.44,2
3,1,0,0,0,3,7,runail,0.40,0
4,0,0,1,0,4,8,metzger,3.33,3
...,...,...,...,...,...,...,...,...,...
895202,0,0,0,1,41,20,grattol,5.24,112934
895203,0,0,0,1,601,225,irisk,1.43,126974
895204,0,0,0,1,29,33,cosmoprofi,6.35,175060
895205,1,0,0,0,223,113,severina,3.16,48925


Converting all the event types into binary variables

In [13]:
cosmetic = cosdummy.drop(['brand','category_id','event_type_view','event_type_remove_from_cart','event_type_cart','price'], axis = 1)
cosmetic

,event_type_purchase,product_id,user_id
0,0,0,0
1,0,1,1
2,0,2,2
3,0,3,0
4,0,4,3
...,...,...,...
895202,0,41,112934
895203,0,601,126974
895204,0,29,175060
895205,0,223,48925


In [14]:
table = pd.pivot_table(cosmetic, values='event_type_purchase', index=['user_id'],
                    columns=['product_id'] , aggfunc= sum, fill_value=0)
table

product_id,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
user_id,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180879,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Computer is not powerful enough to run the entire file so I used a subsection of it for analysis
cosmetic1 = cosdummy[:500]

In [16]:
cosmetic1
cosmetic1["brand"] = cosmetic1["brand"].astype('category')
cosmetic1.dtypes

<ipython-input-16-986c822cb699>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosmetic1["brand"] = cosmetic1["brand"].astype('category')


event_type_cart                   uint8
event_type_purchase               uint8
event_type_remove_from_cart       uint8
event_type_view                   uint8
product_id                        int64
category_id                       int64
brand                          category
price                           float64
user_id                           int64
dtype: object

In [17]:
cosmetic1


,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view,product_id,category_id,brand,price,user_id
0,1,0,0,0,0,2,runail,1.19,0
1,0,0,0,1,1,4,runail,2.62,1
2,0,0,0,1,2,5,ingarden,4.44,2
3,1,0,0,0,3,7,runail,0.40,0
4,0,0,1,0,4,8,metzger,3.33,3
...,...,...,...,...,...,...,...,...,...
495,0,1,0,0,186,6,roubloff,2.52,83
496,0,1,0,0,217,18,runail,0.40,83
497,0,1,0,0,219,18,runail,0.40,83
498,0,1,0,0,10,6,roubloff,1.71,83


In [18]:
#grouping by the user and product ID to create a ratings dataframe for future usage
ratings = cosmetic1.groupby(['user_id', 'product_id']).sum().reset_index()
ratings

,user_id,product_id,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view,category_id,price
0,0,0,1,1,0,0,4,2.38
1,0,3,1,1,0,0,14,0.80
2,0,7,2,1,0,0,33,8.58
3,0,13,2,1,0,1,76,5.72
4,0,32,1,1,0,0,6,3.18
...,...,...,...,...,...,...,...,...
312,117,235,0,0,0,1,62,12.22
313,118,123,0,0,0,1,23,44.29
314,119,237,1,0,0,0,135,1.51
315,120,238,0,0,2,0,124,4.76


In [19]:
#grabbing the mean ratings of all users
mean = ratings.groupby(by = 'user_id', as_index = False)['event_type_purchase'].mean()
mean

,user_id,event_type_purchase
0,0,1.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
117,117,0.0
118,118,0.0
119,119,0.0
120,120,0.0


In [20]:
products = cosmetic1.groupby(['product_id', 'brand'])['category_id'].count().reset_index()
products = pd.DataFrame(data = products)
products

,product_id,brand,category_id
0,0,art-visage,0
1,0,balbcare,0
2,0,bluesky,0
3,0,bpw.style,0
4,0,cnd,0
...,...,...,...
11035,239,strong,0
11036,239,thuya,0
11037,239,uno,0
11038,239,yoko,0


In [21]:
products.dtypes

product_id        int64
brand          category
category_id       int64
dtype: object

Getting the brand and product IDs for recommender system in the future.

In [22]:

# pivot table of values
final = pd.pivot_table(cosmetic1, values='event_type_purchase', index=['user_id'],
                    columns=['product_id'])
final

product_id,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.5,NaN,NaN,0.5,NaN,NaN,NaN,0.333333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [23]:
#replacing Nan with product average
final_product = final.fillna(final.mean(axis = 0))
final_product

product_id,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.50,0.0,0.0,0.5,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,1.0
1,0.25,0.0,0.0,0.5,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,1.0
2,0.25,0.0,0.0,0.5,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,1.0
3,0.25,0.0,0.0,0.5,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,1.0
4,0.25,0.0,0.0,0.5,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,0.25,0.0,0.0,0.5,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,1.0
118,0.25,0.0,0.0,0.5,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,1.0
119,0.25,0.0,0.0,0.5,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,1.0


In [24]:
#replacing Nan with user average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis =1)
final_user

product_id,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.5,0.52381,0.52381,0.5,0.52381,0.52381,0.52381,0.333333,0.52381,0.52381,...,0.52381,0.52381,0.52381,0.52381,0.52381,0.52381,0.52381,0.52381,0.52381,0.52381
1,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
118,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
119,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [25]:
from sklearn.metrics.pairwise import cosine_similarity

# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.990351,0.0,0.0,0.0,...,0.0,0.0,0.995139,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
cosine = cosine_similarity(final_product)
np.fill_diagonal(cosine, 0 )
similarity_with_product = pd.DataFrame(cosine,index=final_product.index)
similarity_with_product.columns=final_user.index
similarity_with_product.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.979016,0.979016,0.970878,0.974248,0.979016,0.979016,0.976337,0.979016,0.979016,...,0.979016,0.979016,0.960084,0.979016,0.975670,0.979016,0.979016,0.979016,0.979016,0.979016
1,0.979016,0.000000,1.000000,0.998962,0.994882,1.000000,1.000000,0.997124,1.000000,1.000000,...,1.000000,1.000000,0.981136,1.000000,0.999681,1.000000,1.000000,1.000000,1.000000,1.000000
2,0.979016,1.000000,0.000000,0.998962,0.994882,1.000000,1.000000,0.997124,1.000000,1.000000,...,1.000000,1.000000,0.981136,1.000000,0.999681,1.000000,1.000000,1.000000,1.000000,1.000000
3,0.970878,0.998962,0.998962,0.000000,0.993828,0.998962,0.998962,0.996078,0.998962,0.998962,...,0.998962,0.998962,0.980159,0.998962,0.999281,0.998962,0.998962,0.998962,0.998962,0.998962
4,0.974248,0.994882,0.994882,0.993828,0.000000,0.994882,0.994882,0.991961,0.994882,0.994882,...,0.994882,0.994882,0.956580,0.994882,0.994558,0.994882,0.994882,0.994882,0.994882,0.994882


Now we have two different dataframes with user and product similarity

In [27]:
#finding the most similar neighbors based on a user or product criterion
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['Closest {} Neighbor'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [28]:
# top 10 neighbours for each user
sim_user_10_u = find_n_neighbours(similarity_with_user,10)
sim_user_10_u.head()

,Closest 1 Neighbor,Closest 2 Neighbor,Closest 3 Neighbor,Closest 4 Neighbor,Closest 5 Neighbor,Closest 6 Neighbor,Closest 7 Neighbor,Closest 8 Neighbor,Closest 9 Neighbor,Closest 10 Neighbor
user_id,,,,,,,,,,
0,49,114,30,6,83,71,24,35,34,42
1,121,30,32,33,34,35,36,37,38,39
2,121,30,32,33,34,35,36,37,38,39
3,121,30,32,33,34,35,36,37,38,39
4,121,30,32,33,34,35,36,37,38,39


In [29]:
# top 10 neighbours for each user
sim_user_10_p = find_n_neighbours(similarity_with_product,10)
sim_user_10_p.head()

,Closest 1 Neighbor,Closest 2 Neighbor,Closest 3 Neighbor,Closest 4 Neighbor,Closest 5 Neighbor,Closest 6 Neighbor,Closest 7 Neighbor,Closest 8 Neighbor,Closest 9 Neighbor,Closest 10 Neighbor
user_id,,,,,,,,,,
0,121,64,36,37,38,39,40,41,42,43
1,121,65,37,38,39,40,41,42,43,44
2,121,65,37,38,39,40,41,42,43,44
3,54,15,116,86,121,46,36,37,38,39
4,121,64,36,37,38,39,40,41,42,43


Getting closest neighbors, chose 10 to begin with so the dataframes do not get cluttered

In [30]:
ratings = ratings.astype({"product_id": str})
shop_user = ratings.groupby(by = 'user_id')['product_id'].apply(lambda x:','.join(x))

In [31]:
def User_item_score1(user):
    product_seen_by_user = final.columns[final[final.index==user].notna().any()].tolist() #creating a dataframe of all items seen by the user
    a = sim_user_10_p[sim_user_10_p.index==user].values #based off of the neighbors we get the values
    b = a.squeeze().tolist()
    d = shop_user[shop_user.index.isin(b)]
    l = ','.join(d.values)
    product_seen_by_similar_users = l.split(',')
    product_under_consideration = list(set(product_seen_by_similar_users)-set(list(map(str, product_seen_by_user))))
    product_under_consideration = list(map(int, product_under_consideration))
    #mapping and listing products for usage
    score = []
    for item in product_under_consideration: #beginning to loop through all the products 
        c = final_product.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = mean.loc[mean['user_id'] == user,'event_type_purchase'].values[0] #getting the users age based on product
        index = f.index.values.squeeze().tolist() #adding to to a list
        corr = similarity_with_product.loc[user,index] 
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1) #using the average score and correlation for a new list
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno) #final score 
        score.append(final_score)
    data = pd.DataFrame({'product_id':product_under_consideration,'score':score}) #creating the dataframe and outputting it
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    product_Name = top_5_recommendation.merge(products, how='inner', on='product_id') #getting the product recommendations
    product_brand = product_Name.product_id.tolist()#   
  
    return product_brand

In [32]:
user = int(input("Enter the user id to whom you want to recommend : ")) #making it so its a bit more interactive
predicted_product = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : ", user)
print("   ")
for i in predicted_product: #displaying product recommendations
    print(i)

Enter the user id to whom you want to recommend : 33
 
The Recommendations for User Id :  33
   
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104


System displays top 5 products for each user based on previous users.

In [33]:
N = final_user.as_matrix() #using the user-user matrix to do matrix factorization
N2 = np.array(final_user)
print(N.shape)

AttributeError: 'DataFrame' object has no attribute 'as_matrix'

In [ ]:
N

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(N2, test_size=0.2, random_state=42)

In [ ]:
#matrix factorization with scikit learn
import numpy as np
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(x_train)
H = model.components_
print('factorized matrix', W) #new matrix
print(W.shape)
print('model components', H)

In [ ]:

totCount = 0
totError = 0
for u in range(97): #had to use the shape of W for this to work 
    err_u = 0
    rateCount_u = 1
    for j in range(1): #with it only looping into the second column 
        if (N2[u,j] > 0): ### Only use known ratings computing error
            rateCount_u =+ 1
            err_u += abs((W[u,j]) - N2[u,j]) #getting absolute error

    print("Mean Absolute Error for User %d = %0.3f" %(u, err_u/rateCount_u)) 
    totCount += rateCount_u
    totError += err_u
print
print("Overall Mean Absolute Error = %0.3f" %(totError/totCount)) #total absolute error

Validation of the recommendations system using Mean Absolute Error, in comparison to the actual matrix vs the matrix that was factorized. 